In [1]:
!pip install PyMySQL

  Using cached PyMySQL-0.9.3-py2.py3-none-any.whl (47 kB)


In [2]:
!pip install flask_login

  Using cached Flask_Login-0.5.0-py2.py3-none-any.whl (16 kB)


In [3]:
!pip install flask_cors

  Using cached Flask_Cors-3.0.8-py2.py3-none-any.whl (14 kB)


In [ ]:
from flask import Flask, render_template, url_for, redirect, request, make_response, Response, jsonify
from flask_login import LoginManager, login_user, logout_user, login_required, UserMixin
from flask_cors import CORS

import pymysql

app = Flask(__name__)
CORS(app)

connection = pymysql.connect(host="localhost", 
                             user="root",
                             passwd="root", 
                             db="lesson5",
                             charset='utf8mb4')

login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = 'login'
app.config['SECRET_KEY'] = "lkkajdghdadkglajkgah"

@login_manager.user_loader
def load_user(user_id):
    return User(user_id)

class User(UserMixin):
  def __init__(self,id):
    self.id = id

############ Web Page Routes Setup ###############

@app.route("/")
@login_required
def home():
    return render_template('home.html')

@app.route("/login")
def login():
    message = 'Please login in first.'
    return render_template('login.html', message=message)

@app.route('/query_auth')
def query_auth():
    username = request.args.get("username")
    password = request.args.get("password")
    if username == "bernard@cuhk.edu.hk" and password == "1234":
        login_user(User(1))
        message = "Dear " + username + ", welcome to Sameen's pages. Your login has been granted."
        return make_response(message, 200, )
    else:
        message = 'Wrong password!'
        return make_response( message,401,{'WWW-Authenticate':'Basic realm="Login Required"'})

@app.route("/form_auth",methods=['POST'])
def form_auth():
    username = request.form['email']
    password = request.form['pwd']
    if username == "bernard@cuhk.edu.hk" and password == "1234":
        login_user(User(1))
        message = "Dear " + username + ", welcome to Sameen's pages. Your login has been granted."
        return render_template('home.html', message=message)
    else:
        message = 'Wrong password!'
        return render_template('login.html',message=message)

@app.route('/album')
@login_required
def album():
    user = {"name":"Bernard"}
    with connection.cursor() as cursor:
        sql = "SELECT * FROM poster"
        cursor.execute(sql)
        result = cursor.fetchall()
    final_result = [list(i) for i in result]
    dataset=[]
    dict={}
    for i in final_result:
        dict['Name'] = i[0]
        dict['desc'] = i[1]
        dict['Year'] = i[2]
        dict['Cast'] = i[3]
        dict['Type'] = i[4]
        dict['url'] = i[5]
        dataset.append(dict.copy()) #markers.append(fld.copy())
    return render_template('product2.html',album_user=user, dataset=dataset)

@app.route('/logout/')
@login_required
def logout():
    logout_user()
    message = 'Thanks for logging out.'
    return render_template('login.html',message=message)

@app.errorhandler(500)
def internal_error(error):
    message = 'Wrong password!'
    return render_template('login.html',message=message),500

######### API Endpoints ##########

@app.route('/api_auth', methods=['POST'])
def api_auth():
    username = request.json['username']
    password = request.json['password']
    if username == "bernard@cuhk.edu.hk" and password == "1234":
        login_user(User(1))
        message = "Dear " + username + ", welcome to Sameen's pages. Your login has been granted."
        return jsonify({'response':'ok!','message':message})
    else:
        message = 'Wrong password!'
        return jsonify({'response':'Invalid!','message':'Cannot authenticate.'})
@app.route('/api_album', methods=['POST'])
@login_required
def api_album():
    with connection.cursor() as cursor:
        sql = "SELECT * FROM poster"
        cursor.execute(sql)
        result = cursor.fetchall()
    final_result = [list(i) for i in result]
    dataset=[]
    dict={}
    for i in final_result:
        dict['Name'] = i[0]
        dict['desc'] = i[1]
        dict['Year'] = i[2]
        dict['Cast'] = i[3]
        dict['Type'] = i[4]
        dict['url'] = i[5]
        dataset.append(dict.copy()) #markers.append(fld.copy())
    return jsonify({'Album': dataset})

######### Run Flask Web App at Port 9030 ##########

if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 9030, app)

 * Running on http://localhost:9030/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Mar/2020 21:54:08] "GET / HTTP/1.1" 302 -
127.0.0.1 - - [22/Mar/2020 21:54:09] "GET /login?next=%2F HTTP/1.1" 200 -
127.0.0.1 - - [22/Mar/2020 21:54:09] "GET /static/css/bootstrap.min.css HTTP/1.1" 200 -
127.0.0.1 - - [22/Mar/2020 21:54:09] "GET /static/js/jquery.min.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Mar/2020 21:54:09] "GET /static/css/font-awesome.min.css HTTP/1.1" 200 -
127.0.0.1 - - [22/Mar/2020 21:54:09] "GET /static/css/bootstrap.css HTTP/1.1" 200 -
127.0.0.1 - - [22/Mar/2020 21:54:09] "GET /static/css/latofont.css HTTP/1.1" 200 -
127.0.0.1 - - [22/Mar/2020 21:54:09] "GET /static/css/mystyle.css HTTP/1.1" 200 -
127.0.0.1 - - [22/Mar/2020 21:54:09] "GET /static/css/button.css HTTP/1.1" 200 -
127.0.0.1 - - [22/Mar/2020 21:54:09] "GET /static/js/bootstrap.min.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Mar/2020 21:54:09] "GET /static/fonts/glyphicons-halflings-regular.woff2 HTTP/1.1" 200 -
127.0.0.1 - - [22/Mar